In [52]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings('ignore')

In [53]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

In [54]:
# Exploring High school directory table

query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


In [55]:
# Number of schools in each borough

query = """
SELECT borough, COUNT(Distinct dbn) AS total_schools
FROM nyc_schools.high_school_directory
GROUP BY borough;"""

df = pd.read_sql(query, conn)
df.head()

,borough,total_schools
0,Bronx,118
1,Brooklyn,121
2,Manhattan,106
3,Queens,80
4,Staten Island,10


In [56]:
# Exploring data in School demographics table

query = "SELECT * FROM nyc_schools.school_demographics LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

,dbn,Name,schoolyear,fl_percent,frl_percent,total_enrollment,prek,k,grade1,grade2,...,black_num,black_per,hispanic_num,hispanic_per,white_num,white_per,male_num,male_per,female_num,female_per
0,01M015,P.S. 015 ROBERTO CLEMENTE,20052006,89.4,NaN,281,15,36,40,33,...,74,26.3,189,67.3,5,1.8,158,56.2,123,43.8
1,01M015,P.S. 015 ROBERTO CLEMENTE,20062007,89.4,NaN,243,15,29,39,38,...,68,28.0,153,63.0,4,1.6,140,57.6,103,42.4
2,01M015,P.S. 015 ROBERTO CLEMENTE,20072008,89.4,NaN,261,18,43,39,36,...,77,29.5,157,60.2,7,2.7,143,54.8,118,45.2
3,01M015,P.S. 015 ROBERTO CLEMENTE,20082009,89.4,NaN,252,17,37,44,32,...,75,29.8,149,59.1,7,2.8,149,59.1,103,40.9
4,01M015,P.S. 015 ROBERTO CLEMENTE,20092010,,96.5,208,16,40,28,32,...,67,32.2,118,56.7,6,2.9,124,59.6,84,40.4


In [61]:
# Average % of English Language Learners

query = """
Select hd.borough,Avg(sd.ell_percent) AS avg_ell_precentage
From nyc_schools.high_school_directory As hd
LEFT JOIN nyc_schools.school_demographics As sd
On hd.dbn = sd.dbn
Group By hd.borough;"""

df = pd.read_sql(query, conn)
df.head()


,borough,avg_ell_precentage
0,Brooklyn,NaN
1,Queens,NaN
2,Staten Island,NaN
3,Manhattan,7.5725
4,Bronx,NaN


In [60]:
#Top 3 Schools with hightest percentage of special education.


query = """SELECT borough, school_name, sped_percent
FROM (
    SELECT
        hd.borough,
        hd.school_name,
        sd.sped_percent,
        ROW_NUMBER() OVER (PARTITION BY hd.borough ORDER BY sd.sped_percent DESC) as rank
    From nyc_schools.high_school_directory As hd
    LEFT JOIN nyc_schools.school_demographics As sd
    ON hd.dbn = sd.dbn
    WHERE sd.sped_percent IS NOT NULL
) As ranked
WHERE rank <= 3
ORDER BY borough, rank""";

df = pd.read_sql(query, conn)
df.head()



,borough,school_name,sped_percent
0,Manhattan,East Side Community School,28.8
1,Manhattan,East Side Community School,27.7
2,Manhattan,East Side Community School,26.7


Observations:


* English language learners are only in schools in  
  Manhattan as per the results.
   

*  Also there is only one school in Manhattan   
   providing special education. However it is showing three different values.  





